In [1]:
import gc
import sys
sys.path.append("../../pytorch-forecasting")
sys.path.append("../")

import pytorch_lightning as pl
import seaborn as sns
import torch
from data_factory.dataLoader import StockPricesLoader
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import notebook

from utilities import *
import pandas as pd
from tqdm import tqdm

from data_factory.prepared_data import TimeSeriesData, PreparedData

sns.set_style("whitegrid")
tqdm.pandas()

pl.seed_everything(42)

Global seed set to 42


42

In [2]:
config = load_config("../config/config.yml")
assert config['model'] == 'temporal_fusion_transformer', 'Invalid model in file configuration for this script'
model = config['model']

data_ts: TimeSeriesData = TimeSeriesData.from_file('../data/save/timeseries_data.pkl')

#### Create Data loader

In [3]:
batch_size = config[model]['sliding_window']['batch_size']

# Training
train_dl = data_ts.train.to_dataloader(train=True, batch_size=batch_size, num_workers=12)


# Validation
val_dl = data_ts.val.to_dataloader(train=False, batch_size=batch_size, num_workers=12, shuffle=False)


# Testing
test_dl = data_ts.test.to_dataloader(
    batch_size=data_ts.test_set_size,
    num_workers=12,
    shuffle=False
)

## TemporalFusionTransformer

In [5]:
from pytorch_forecasting.models import TemporalFusionTransformer
import logging
from logging import WARNING
logging.basicConfig(level=WARNING)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")


args = dict(
    hidden_size=config['temporal_fusion_transformer']['hidden_size'],
    lstm_layers=config['temporal_fusion_transformer']['lstm_layers'],
    dropout=config['temporal_fusion_transformer']['dropout'],
    attention_head_size=config['temporal_fusion_transformer']['attention_head_size'],
    output_size=config['temporal_fusion_transformer']['output_size'],
)

trainer = pl.Trainer(
    accelerator='gpu',
    gradient_clip_val=0.1,
    auto_lr_find=True,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    weights_summary="top",
    max_epochs=5
)

model = TemporalFusionTransformer.from_dataset(
    data_ts.train,
    **args
)

# model = TemporalFusionTransformer.load_from_checkpoint('./lightning_logs/lightning_logs/version_41/checkpoints/epoch=0-step=46790.ckpt')

print(f"Number of parameters in network: {model.size() / 1e3:.1f}k")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 74.2k


#### Train the model

In [6]:
trainer.validate(model, val_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_MAE            1.1724119186401367
        val_MAPE             187.1892547607422
        val_RMSE            1.3325738906860352
        val_SMAPE            1.508351445198059
        val_loss            0.6218529343605042
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.6218529343605042,
  'val_SMAPE': 1.508351445198059,
  'val_MAE': 1.1724119186401367,
  'val_RMSE': 1.3325738906860352,
  'val_MAPE': 187.1892547607422}]

In [7]:
trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 32.5 K
3  | prescalers                         | ModuleDict                      | 496   
4  | static_variable_selection          | VariableSelectionNetwork        | 428   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 22.3 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 662   
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_lstm 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/cyprien/anaconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [8]:
trainer.validate(model, val_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_MAE            0.18998563289642334
        val_MAPE             38.63682556152344
        val_RMSE            0.25938352942466736
        val_SMAPE           0.2665172219276428
        val_loss            0.06368018686771393
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.06368018686771393,
  'val_SMAPE': 0.2665172219276428,
  'val_MAE': 0.18998563289642334,
  'val_RMSE': 0.25938352942466736,
  'val_MAPE': 38.63682556152344}]

In [9]:
trainer.save_checkpoint('modelv3.ckpt')

In [ ]:
Loss at start : 0.0103